# History match the freyberg model using ``K`` parameter zones

In [ ]:
%matplotlib inline
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyemu
import platform
import pestools as pt
if 'window' in platform.platform().lower():
    ppp = 'pest++'
else:
    ppp = './pestpp'

In [ ]:
base_dir = os.path.join("..","..","models","Freyberg","Freyberg_zones")
assert os.path.exists(base_dir)
[shutil.copy2(os.path.join(base_dir,f),f) for f in os.listdir(base_dir)];

# Let's set up a model using multiple zones for Hydraulic Conductivity

We have an interpretation of zonation, say from a geologic lithology map.

In [ ]:
inzones = np.loadtxt('hk.zones')

In [ ]:
plt.imshow(inzones,interpolation='nearest',cmap='viridis')
plt.colorbar()

In [ ]:
# see what the unique values are
np.unique(inzones)

## Look at `hk.zones` in a text editor
How can we make a template file to handle this?

In [ ]:
pyemu.utils.start_slaves(base_dir,"{0}".format(ppp),"freyberg_zones.pst",20,slave_root='.',port=4004,master_dir='.')


Let's explore the results

In [ ]:
df_obj = pd.read_csv("freyberg_zones.iobj",index_col=0)
df_obj

In [ ]:
res = pt.Res('freyberg_zones.rei')

In [ ]:
res.describe_groups('head_cal')

In [ ]:
res.describe_groups('flux_cal')

In [ ]:
res.plot_one2one('head_cal',print_stats=['Mean', 'MAE', 'RMSE'])

In [ ]:
res.plot_measured_vs_residual('head_cal')

In [ ]:
df_paru = pd.read_csv("freyberg_zones.par.usum.csv")
df_paru

In [ ]:
df_predu = pd.read_csv("freyberg_zones.pred.usum.csv",index_col=0)
df_predu

In [ ]:
df_foreu_kr = pd.read_csv(os.path.join("freybergkr.pred.usum.csv"),index_col=0)
df_foreu_kr.loc[:,"reduction"] = 100.0 *  (1.0 - (df_foreu_kr.post_stdev / df_foreu_kr.prior_stdev))
df_foreu_kr

In [ ]:
df_foreu = pd.read_csv("freyberg_zones.pred.usum.csv",index_col=0)
df_foreu.loc[:,"reduction"] = 100.0 *  (1.0 - (df_foreu.post_stdev / df_foreu.prior_stdev))
df_foreu_concat = pd.concat([df_foreu,df_foreu_kr],join="outer",axis=1,keys=["zones","k+r"])
df_foreu_concat

In [ ]:
for forecast in df_predu.index:
    ax = df_predu.loc[forecast,["prior_stdev","post_stdev"]].plot(kind="bar")
    ax.set_title(forecast)
    plt.show()

# As is our custom, we can make a plot

## (blue is zone, green is k and r homogeneous)

In [ ]:
df_foreu_concat.loc[:,(slice(None),"reduction")].plot(kind="bar", legend=False)
plt.title('Percent REDUCTION of forecast uncertainty')